In [60]:
# -*- coding: utf-8 -*-
import cv2
import sys
from constants import *
from emotion_recognition import EmotionRecognition
from os.path import join
import os
import numpy as np
import pylab
#import matplotlib.pyplot as plt
import tensorflow as tf

import pandas
from pandas import DataFrame
from skimage.transform import resize

#from Udacity
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle

In [18]:
%matplotlib inline
from IPython.core.display import HTML
HTML("""<style> .rendered_html code { 
    padding: 2px 4px;
    color: #c7254e;
    background-color: #f9f2f4;
    border-radius: 4px;
} </style>""")

#Load file in folder
dirname = 'test'

#Load every image file in the provided directory
filenames = [os.path.join(dirname, fname)
             for fname in os.listdir(dirname)]

#Find Happy face only

import glob
happy = glob.glob('RafD/*happy*')
angry = glob.glob('RafD/*angry*')
sad = glob.glob('RafD/*sad*')
contemptuous = glob.glob('RafD/*contemptuous*')
disgusted = glob.glob('RafD/*disgusted*')
neutral = glob.glob('RafD/*neutral*')
fearful = glob.glob('RafD/*fearful*')
surprised = glob.glob('RafD/*surprised*')



#Make sure we have exactly 10 image files for test (you can include more)
happy = happy[:16]
sad = sad[:16]
#assert(len(happy) == 16)
#assert(len(sad) == 16)

from PIL import Image

directory = 'test'
width = 100
height = 100

for image in os.listdir(directory):
    print('Resizing image ' + image)
    # Open the image file
    img = Image.open(os.path.join(directory, image))
    # Convert into Grayscale
    img = img.convert('L')
    # Resize it
    img = img.resize((width, height), Image.BILINEAR)
    # Save it back to disk.
    img.save(os.path.join(directory, 'resized-' + image))

In [159]:
#Save folder list in list
num_classes = 2
np.random.seed(133)

def folder_list(folder_name):
    data_folders = [
        os.path.join(folder_name, d) for d in sorted(os.listdir(folder_name))
            if os.path.isdir(os.path.join(folder_name, d))]
    if len(data_folders) != num_classes:
        raise Exception(
          'Expected %d folders, one per class. Found %d instead.' % (
            num_classes, len(data_folders)))
    return data_folders

In [175]:
train = 'train/'
test = 'test/'

train_folders = folder_list(train)
test_folders = folder_list(test)

In [161]:
#Test image set

import random
import hashlib
%matplotlib inline

def disp_samples(data_folders, sample_size):
    for folder in data_folders:
        print(folder)
        image_files = os.listdir(folder)
        image_sample = random.sample(image_files, sample_size)
        for image in image_sample:
            image_file = os.path.join(folder, image)
            i = Image(filename=image_file)
            display(i)

In [153]:
disp_samples(train_folders, 1)

test/A


TypeError: 'module' object is not callable

In [176]:
image_size = 100  # Pixel width and height.
pixel_depth = 255.0  # Number of levels per pixel.
#width = 1024
#height = 681
#color = 3

In [177]:
def load_letter(folder, min_num_images):
  """Load the data for a single letter label."""
  image_files = os.listdir(folder)
  dataset = np.ndarray(shape=(len(image_files), image_size, image_size),
                         dtype=np.float32)
  image_index = 0
  print(folder)
  for image in os.listdir(folder):
    image_file = os.path.join(folder, image)
    try:
      image_data = (ndimage.imread(image_file).astype(float) - 
                    pixel_depth / 2) / pixel_depth
      if image_data.shape != (image_size, image_size):
        raise Exception('Unexpected image shape: %s' % str(image_data.shape))
      dataset[image_index, :, :] = image_data
      image_index += 1
    except IOError as e:
      print('Could not read:', image_file, ':', e, '- it\'s ok, skipping.')
    
  num_images = image_index
  dataset = dataset[0:num_images, :, :]
  if num_images < min_num_images:
    raise Exception('Many fewer images than expected: %d < %d' %
                    (num_images, min_num_images))
    
  print('Full dataset tensor:', dataset.shape)
  print('Mean:', np.mean(dataset))
  print('Standard deviation:', np.std(dataset))
  return dataset
        
def maybe_pickle(data_folders, min_num_images_per_class, force=False):
  dataset_names = []
  for folder in data_folders:
    set_filename = folder + '.pickle'
    dataset_names.append(set_filename)
    if os.path.exists(set_filename) and not force:
      # You may override by setting force=True.
      print('%s already present - Skipping pickling.' % set_filename)
    else:
      print('Pickling %s.' % set_filename)
      dataset = load_letter(folder, min_num_images_per_class)
      try:
        with open(set_filename, 'wb') as f:
          pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
      except Exception as e:
        print('Unable to save data to', set_filename, ':', e)
  
  return dataset_names

train_datasets = maybe_pickle(train_folders, 8)
test_datasets = maybe_pickle(test_folders, 8)

Pickling train/angry.pickle.
train/angry
Full dataset tensor: (10, 100, 100)
Mean: 0.110281
Standard deviation: 0.274469
Pickling train/sad.pickle.
train/sad
Full dataset tensor: (8, 100, 100)
Mean: 0.106049
Standard deviation: 0.278165
Pickling test/angry.pickle.
test/angry
Full dataset tensor: (10, 100, 100)
Mean: 0.110281
Standard deviation: 0.274469
Pickling test/sad.pickle.
test/sad
Full dataset tensor: (8, 100, 100)
Mean: 0.106049
Standard deviation: 0.278165


In [178]:
#we expect the data to be balanced across classes

def disp_number_images(data_folders):
    for folder in data_folders:
        pickle_filename = ''.join(folder) + '.pickle'
        try:
            with open(pickle_filename, 'r') as f:
                dataset = pickle.load(f)
        except Exception as e:
            print('Unable to read data from', pickle_filename, ':', e)
            return
        print('Number of images in ', folder, ' : ', len(dataset))
        
disp_number_images(train_folders)
disp_number_images(test_folders)

Number of images in  train/angry  :  10
Number of images in  train/sad  :  8
Number of images in  test/angry  :  10
Number of images in  test/sad  :  8


In [179]:
def make_arrays(nb_rows, img_size):
  if nb_rows:
    dataset = np.ndarray((nb_rows, img_size, img_size), dtype=np.float32)
    labels = np.ndarray(nb_rows, dtype=np.int32)
  else:
    dataset, labels = None, None
  return dataset, labels

def merge_datasets(pickle_files, train_size, valid_size=0):
  num_classes = len(pickle_files)
  valid_dataset, valid_labels = make_arrays(valid_size, image_size)
  train_dataset, train_labels = make_arrays(train_size, image_size)
  vsize_per_class = valid_size // num_classes
  tsize_per_class = train_size // num_classes
    
  start_v, start_t = 0, 0
  end_v, end_t = vsize_per_class, tsize_per_class
  end_l = vsize_per_class+tsize_per_class
  for label, pickle_file in enumerate(pickle_files):       
    try:
      with open(pickle_file, 'rb') as f:
        letter_set = pickle.load(f)
        # let's shuffle the letters to have random validation and training set
        np.random.shuffle(letter_set)
        if valid_dataset is not None:
          valid_letter = letter_set[:vsize_per_class, :, :]
          valid_dataset[start_v:end_v, :, :] = valid_letter
          valid_labels[start_v:end_v] = label
          start_v += vsize_per_class
          end_v += vsize_per_class
                    
        train_letter = letter_set[vsize_per_class:end_l, :, :]
        train_dataset[start_t:end_t, :, :] = train_letter
        train_labels[start_t:end_t] = label
        start_t += tsize_per_class
        end_t += tsize_per_class
    except Exception as e:
      print('Unable to process data from', pickle_file, ':', e)
      raise
    
  return valid_dataset, valid_labels, train_dataset, train_labels

train_size = 5
valid_size = 3
test_size = 3

valid_dataset, valid_labels, train_dataset, train_labels = merge_datasets(
  train_datasets, train_size, valid_size)
_, _, test_dataset, test_labels = merge_datasets(test_datasets, test_size)

print('Training:', train_dataset.shape, train_labels.shape)
print('Validation:', valid_dataset.shape, valid_labels.shape)
print('Testing:', test_dataset.shape, test_labels.shape)

Training: (5, 100, 100) (5,)
Validation: (3, 100, 100) (3,)
Testing: (3, 100, 100) (3,)


In [185]:
train_labels

array([        0,         0,         1,         1, 340419952], dtype=int32)

In [186]:
def randomize(dataset, labels):
  permutation = np.random.permutation(labels.shape[0])
  shuffled_dataset = dataset[permutation,:,:]
  shuffled_labels = labels[permutation]
  return shuffled_dataset, shuffled_labels
train_dataset, train_labels = randomize(train_dataset, train_labels)
test_dataset, test_labels = randomize(test_dataset, test_labels)
valid_dataset, valid_labels = randomize(valid_dataset, valid_labels)

In [187]:
#Draw histogram
plt.hist(train_labels, 10)
plt.axis([0, 9, 0, 30000])
plt.grid(True)
plt.show()

RuntimeError: Could not create write struct

In [188]:
pickle_file = 'notMNIST.pickle'

try:
  f = open(pickle_file, 'wb')
  save = {
    'train_dataset': train_dataset,
    'train_labels': train_labels,
    'valid_dataset': valid_dataset,
    'valid_labels': valid_labels,
    'test_dataset': test_dataset,
    'test_labels': test_labels,
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print('Unable to save data to', pickle_file, ':', e)
  raise

In [189]:
statinfo = os.stat(pickle_file)
print('Compressed pickle size:', statinfo.st_size)

Compressed pickle size: 440466


In [190]:
statinfo

posix.stat_result(st_mode=33204, st_ino=663734, st_dev=2054, st_nlink=1, st_uid=1000, st_gid=1000, st_size=440466, st_atime=1473747739, st_mtime=1473747739, st_ctime=1473747739)

In [24]:
# Read every filename as an RGB image
imgs_happy = [plt.imread(fname)[..., :3] for fname in happy]
imgs_sad = [plt.imread(fname)[..., :3] for fname in sad]

In [25]:
def imcrop_tosquare(img):
    """Make any image a square image.
    Parameters
    ----------
    img : np.ndarray
        Input image to crop, assumed at least 2d.
    Returns
    -------
    crop : np.ndarray
        Cropped image.
    """
    size = np.min(img.shape[:2])
    extra = img.shape[:2] - size
    crop = img
    for i in np.flatnonzero(extra):
        crop = np.take(crop, extra[i] // 2 + np.r_[:size], axis=i)
    return crop

def montage(images, saveto='montage.png'):
    """Draw all images as a montage separated by 1 pixel borders.
    Also saves the file to the destination specified by `saveto`.
    Parameters
    ----------
    images : numpy.ndarray
        Input array to create montage of.  Array should be:
        batch x height x width x channels.
    saveto : str
        Location to save the resulting montage image.
    Returns
    -------
    m : numpy.ndarray
        Montage image.
    """
    if isinstance(images, list):
        images = np.array(images)
    img_h = images.shape[1]
    img_w = images.shape[2]
    n_plots = int(np.ceil(np.sqrt(images.shape[0])))
    if len(images.shape) == 4 and images.shape[3] == 3:
        m = np.ones(
            (images.shape[1] * n_plots + n_plots + 1,
             images.shape[2] * n_plots + n_plots + 1, 3)) * 0.5
    else:
        m = np.ones(
            (images.shape[1] * n_plots + n_plots + 1,
             images.shape[2] * n_plots + n_plots + 1)) * 0.5
    for i in range(n_plots):
        for j in range(n_plots):
            this_filter = i * n_plots + j
            if this_filter < images.shape[0]:
                this_img = images[this_filter]
                m[1 + i + i * img_h:1 + i + (i + 1) * img_h,
                  1 + j + j * img_w:1 + j + (j + 1) * img_w] = this_img
    plt.imsave(arr=m, fname=saveto)
    return m

In [26]:
# Plot the resulting dataset:
# Make sure you "run" this cell after you create your `imgs` variable as a 4-D array!

plt.figure(figsize=(10, 10))
plt.imshow(montage(imgs_happy, saveto='dataset.png'))

RuntimeError: Could not create write struct

In [32]:
#Cropping, resize and convert to 4d array

def img_preprocessing( img_name ):
    # Crop every image to a square
    imgs = [imcrop_tosquare(img_i) for img_i in img_name]

    # Then resize the square image to 48 x 48 pixels
    imgs = [resize(img_i, (28, 28)) for img_i in imgs]

    # Make list of 3d imgaes a 4d array with the first dimension the number of images:
    imgs = np.array(imgs).astype(np.float32)
    return imgs

happy_pre = img_preprocessing(imgs_happy)
sad_pre = img_preprocessing(imgs_sad)

In [10]:
#img labeling

happy_label = np.zeros((16,2))
happy_label[:,0] = 1 
sad_label = np.zeros((16,2))
sad_label[:,1] = 1 

In [98]:
import cv2
gray_image = cv2.cvtColor(a, cv2.COLOR_BGR2GRAY)
gray_image.shape

(28, 28)

In [95]:
a = happy_pre[0]
a.shape

(28, 28, 3)

In [87]:
d = happy_pre
d = d.reshape([-1, 28, 28, 1])
d.shape

(48, 28, 28, 1)

In [77]:
import tflearn.datasets.mnist as mnist

X, Y, testX, testY = mnist.load_data(one_hot=True)


Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz


In [73]:
X = X.reshape([-1, 28, 28, 1])
testX = testX.reshape([-1, 28, 28, 1])
X.shape

(55000, 28, 28, 1)

In [28]:
example_pic = imgs[0].shape
np_pic = np.array(example_pic)
np_pic

array([48, 48,  3])

In [32]:
image = tf.placeholder(tf.float32)

result = tf.add(image, 0)

h1 = layer(x,w)
h2 = layer(h1,w)
h3 = layer(h2,w)
O = layer(h3, w)

loss = lossfunction( O, label)
train = Optimizer.minimize(loss)

with tf.Session() as sess:
    sess.run( result, feed_dict={x: Image, y= Label})

In [ ]:
image = tf.placeholder(tf.float32)

result = tf.add(image, 0)

with tf.Session() as sess:
    sess.run( result, feed_dict={image: np_pic})

In [59]:
s = pandas.DataFrame(imgs)

ValueError: Must pass 2-d input

In [4]:
network = EmotionRecognition()
network.build_network()

images = np.load('./data_kike.npy')
labels = np.load('./labels_kike.npy')

#images = images.reshape([-1, SIZE_FACE, SIZE_FACE, 1])
#labels = labels.reshape([-1, len(EMOTIONS)])

[+] Building CNN


InternalError: Dst tensor is not initialized.
	 [[Node: Momentum_2/zeros_7 = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [18432,3072] values: 0 0 0...>, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]
Caused by op u'Momentum_2/zeros_7', defined at:
  File "/home/ryan/anaconda2/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/ryan/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/ryan/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/ryan/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/home/ryan/anaconda2/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().start()
  File "/home/ryan/anaconda2/lib/python2.7/site-packages/zmq/eventloop/ioloop.py", line 162, in st

In [19]:
images.shape[0]

13975

In [25]:
data = np.zeros((len(EMOTIONS),len(EMOTIONS)))
for i in xrange(images.shape[0]):
    result = network.predict(images[i])
    data[np.argmax(labels[i]), result[0].index(max(result[0]))] += 1

In [31]:
for i in range(len(data)):
    total = np.sum(data[i])
    for x in range(len(data[0])):
        data[i][x] = data[i][x] / total

print(data)

[[ 0.          0.          0.          0.06584821  0.          0.93415179
   0.        ]
 [ 0.          0.          0.          0.04290429  0.          0.95709571
   0.        ]
 [ 0.          0.          0.          0.04761905  0.          0.95238095
   0.        ]
 [ 0.          0.          0.          0.03048327  0.          0.96951673
   0.        ]
 [ 0.          0.          0.          0.07432854  0.          0.92567146
   0.        ]
 [ 0.          0.          0.          0.03128689  0.          0.96871311
   0.        ]
 [ 0.          0.          0.          0.0678392   0.          0.9321608
   0.        ]]


In [34]:
print '[+] Generating graph'
c = plt.pcolor(data, edgecolors = 'k', linewidths = 4, cmap = 'Blues', vmin = 0.0, vmax = 1.0)
def show_values(pc, fmt="%.2f", **kw):
    from itertools import izip
    pc.update_scalarmappable()
    ax = pc.get_axes()
    ax.set_yticks(np.arange(len(EMOTIONS)) + 0.5, minor = False)
    ax.set_xticks(np.arange(len(EMOTIONS)) + 0.5, minor = False)
    ax.set_xticklabels(EMOTIONS, minor = False)
    ax.set_yticklabels(EMOTIONS, minor = False)
    for p, color, value in izip(pc.get_paths(), pc.get_facecolors(), pc.get_array()):
        x, y = p.vertices[:-2, :].mean(0)
        if np.all(color[:3] > 0.5):
            color = (0.0, 0.0, 0.0)
        else:
            color = (1.0, 1.0, 1.0)
        ax.text(x, y, fmt % value, ha = "center", va = "center", color = color, **kw)

[+] Generating graph


In [35]:
show_values(c)
plt.xlabel('Predicted Emotion')
plt.ylabel('Real Emotion')
plt.show()

/home/ryan/anaconda2/lib/python2.7/site-packages/matplotlib/artist.py:221: MatplotlibDeprecationWarning: This has been deprecated in mpl 1.5, please use the
axes property.  A removal date has not been set.
  warnings.warn(_get_axes_msg, mplDeprecation, stacklevel=1)
